In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from random import *

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping



from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

sns.set(style='white', context='notebook', palette='deep')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [2]:
from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

In [3]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [4]:
#Read  the model architecture from  YAML file
with open('model2.yaml', 'r') as yaml_file:
    yaml_savedModel=yaml_file.read()
# Load the saved Yaml model
import tensorflow as tf
from keras.models import model_from_yaml
model_y= tf.keras.models.model_from_yaml(yaml_savedModel)
model_y.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 32)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 115200)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               29491456  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)              


1.   1 is fake

2.   0 is not fake




In [5]:
def test_it(path):    
    P=[]
    ypath=path
    P.append(array(convert_to_ela_image(ypath, 90).resize((128, 128))).flatten() / 255.0)
    P = np.array(P)
    P = P.reshape(-1, 128, 128, 3)
    P_pred = model_y.predict(P)
    P_pred_classes = np.argmax(P_pred,axis = 1)
    print(P_pred_classes) 
    return P_pred_classes

In [6]:
# uncomment this if you're using google colab
!pip install flask-ngrok

In [7]:
!pip install -U Werkzeug

Requirement already up-to-date: Werkzeug in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [ ]:
# uncomment this if you're using google colab
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename

app = Flask(__name__)

# uncomment this if you're using google colab
run_with_ngrok(app)   #starts ngrok when the app is run
	
@app.route('/', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        f.save(secure_filename(f.filename))
        value = test_it(f.filename)
        print(value)
        
        if value[0] == 1:
          a = "fake"
          b = ""
        else:
          b = "real"
          a =""
        print(f.filename)
        x = "Your image was successfully updloaded and it is "
        return render_template('upload.html', content=x, result1=a, result2=b)
    else:
        return render_template('upload.html')

app.run()		

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3519d2ed5968.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Jul/2020 15:32:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 15:32:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2020 15:33:02] "POST / HTTP/1.1" 200 -


[0]
[0]
0a0ab9a903587d083e7051adfb17a779r.png


127.0.0.1 - - [23/Jul/2020 15:33:17] "POST / HTTP/1.1" 200 -


[0]
[0]
elon.png


127.0.0.1 - - [23/Jul/2020 15:33:27] "POST / HTTP/1.1" 200 -


[0]
[0]
Beutiful.png
